# Poly model
Working on a version of `design_linear.py` that takes a numpy  Polynomial class (or a list of coefficients) instead of a single value to apply linaerly as m * x + 0.0

In [49]:
import numpy as np
import cmm_error_map.design_poly as designp
import cmm_error_map.design_linear as designl
import cmm_error_map.data_cmpts as dc
import cmm_error_map.config.config as cf

In [19]:
model = dc.model_parameters_test.copy()
model_poly = {key: [0.0, value] for key, value in model.items()}

In [21]:
xyz_in = np.array([10.0, 20.0, 30.0])
xyzt = np.array([0.0, 0.0, -250.0])

In [24]:
designl.linear_dependency(xyz_in, model)

{'Txx': 0.000133,
 'Txy': 0.0,
 'Txz': 0.0,
 'Tyx': -0.000224,
 'Tyy': -0.00010180000000000001,
 'Tyz': 0.0,
 'Tzx': 0.00078,
 'Tzy': 0.000138,
 'Tzz': 1.002e-06,
 'Rxx': 7.489999999999999e-08,
 'Rxy': 1.5399999999999998e-07,
 'Rxz': 5e-08,
 'Ryx': -9.16e-08,
 'Ryy': -2.86e-07,
 'Ryz': 4.3800000000000003e-07,
 'Rzx': 7.47e-08,
 'Rzy': -2.382e-08,
 'Rzz': 1.434e-06,
 'Wxy': 0.0,
 'Wxz': 0.0,
 'Wyz': 0.0}

In [27]:
designp.poly_dependency(xyz_in, model)

{'Txx': 0.000133,
 'Txy': 0.0,
 'Txz': 0.0,
 'Tyx': -0.000224,
 'Tyy': -0.00010180000000000001,
 'Tyz': 0.0,
 'Tzx': 0.00078,
 'Tzy': 0.000138,
 'Tzz': 1.002e-06,
 'Rxx': 7.489999999999999e-08,
 'Rxy': 1.5399999999999998e-07,
 'Rxz': 5e-08,
 'Ryx': -9.16e-08,
 'Ryy': -2.86e-07,
 'Ryz': 4.3800000000000003e-07,
 'Rzx': 7.47e-08,
 'Rzy': -2.382e-08,
 'Rzz': 1.434e-06,
 'Wxy': 0.0,
 'Wxz': 0.0,
 'Wyz': 0.0}

In [28]:
designp.model_point(xyz_in, xyzt, model_poly)

array([7.22994912e-04, 5.17014183e-05, 9.60117688e-07])

In [30]:
designp.model_point(xyz_in, xyzt, model)

array([7.22994912e-04, 5.17014183e-05, 9.60117688e-07])

In [31]:
designl.linear_model_point(xyz_in, xyzt, model)

array([7.22994912e-04, 5.17014183e-05, 9.60117688e-07])

In [ ]:
model_params = model_poly
coeffs = {
    key: value if isinstance(value, list) else [0.0, value]
    for key, value in model_params.items()
}
coeffs

{'Txx': [0.0, 1.33e-05],
 'Txy': [0.0, 0.0],
 'Txz': [0.0, 0.0],
 'Tyx': [0.0, -1.12e-05],
 'Tyy': [0.0, -5.09e-06],
 'Tyz': [0.0, 0.0],
 'Tzx': [0.0, 2.6e-05],
 'Tzy': [0.0, 4.6e-06],
 'Tzz': [0.0, 3.34e-08],
 'Rxx': [0.0, 7.49e-09],
 'Rxy': [0.0, 1.54e-08],
 'Rxz': [0.0, 5e-09],
 'Ryx': [0.0, -4.58e-09],
 'Ryy': [0.0, -1.43e-08],
 'Ryz': [0.0, 2.19e-08],
 'Rzx': [0.0, 2.49e-09],
 'Rzy': [0.0, -7.94e-10],
 'Rzz': [0.0, 4.78e-08],
 'Wxy': [0.0, 0.0],
 'Wxz': [0.0, 0.0],
 'Wyz': [0.0, 0.0]}

## profiling
`design_poly.poly_dependency`

In [32]:
from numpy.polynomial import Polynomial

In [34]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [36]:
def pd1(
    xyz: np.ndarray,
    model_params: dict[str, float | list[float]],
) -> dict[str, list[float]]:
    """
    if model params[key] is a single value return a linear dependent
    create a polynomial 0.0 * value * xyz[dependent_axis], that is
    parameter * axis position value for the  axis the parameter is dependent on
    if model params[key] is a list of values
    calculate a polynomial form these coeficients and return
    the polnominal evaluated at the axis value for the dependent axis
    """
    x, y, z = xyz
    ld_dict = {
        "Txx": x,
        "Txy": x,
        "Txz": x,
        "Tyx": y,
        "Tyy": y,
        "Tyz": y,
        "Tzx": z,
        "Tzy": z,
        "Tzz": z,
        "Rxx": x,
        "Rxy": x,
        "Rxz": x,
        "Ryx": y,
        "Ryy": y,
        "Ryz": y,
        "Rzx": z,
        "Rzy": z,
        "Rzz": z,
        "Wxy": x,
        "Wxz": y,
        "Wyz": z,
    }
    # pl = {}
    # for key, value in model_params.items():
    #     if isinstance(value, list):
    #         poly = Polynomial(value)
    #     else:
    #         poly = Polynomial([0.0, value])
    #     pl[key] = poly(ld_dict[key])
    coeffs = {
        key: value if isinstance(value, list) else [0.0, value]
        for key, value in model_params.items()
    }
    pl = {key: Polynomial(coeffs[key])(ld_dict[key]) for key in model_params}
    return pl

In [37]:
%lprun -f pd1 pd1(xyz_in, model)

Timer unit: 1e-09 s

Total time: 0.00079638 s
File: /tmp/ipykernel_21621/1854337976.py
Function: pd1 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def pd1(
     2                                               xyz: np.ndarray,
     3                                               model_params: dict[str, float | list[float]],
     4                                           ) -> dict[str, list[float]]:
     5                                               """
     6                                               if model params[key] is a single value return a linear dependent
     7                                               create a polynomial 0.0 * value * xyz[dependent_axis], that is
     8                                               parameter * axis position value for the  axis the parameter is dependent on
     9                                               if model params[key] is a list of values
    10

In [38]:
def pd2(
    xyz: np.ndarray,
    model_params: dict[str, float | list[float]],
) -> dict[str, list[float]]:
    """
    if model params[key] is a single value return a linear dependent
    create a polynomial 0.0 * value * xyz[dependent_axis], that is
    parameter * axis position value for the  axis the parameter is dependent on
    if model params[key] is a list of values
    calculate a polynomial form these coeficients and return
    the polnominal evaluated at the axis value for the dependent axis
    """
    x, y, z = xyz
    ld_dict = {
        "Txx": x,
        "Txy": x,
        "Txz": x,
        "Tyx": y,
        "Tyy": y,
        "Tyz": y,
        "Tzx": z,
        "Tzy": z,
        "Tzz": z,
        "Rxx": x,
        "Rxy": x,
        "Rxz": x,
        "Ryx": y,
        "Ryy": y,
        "Ryz": y,
        "Rzx": z,
        "Rzy": z,
        "Rzz": z,
        "Wxy": x,
        "Wxz": y,
        "Wyz": z,
    }
    # pl = {}
    # for key, value in model_params.items():
    #     if isinstance(value, list):
    #         poly = Polynomial(value)
    #     else:
    #         poly = Polynomial([0.0, value])
    #     pl[key] = poly(ld_dict[key])
    coeffs = {
        key: value if isinstance(value, list) else [0.0, value]
        for key, value in model_params.items()
    }
    polys = {key: Polynomial(coeffs[key]) for key in model_params}
    pl = {key: polys[key](ld_dict[key]) for key in model_params}
    return pl

In [39]:
%lprun -f pd2 pd2(xyz_in, model)

Timer unit: 1e-09 s

Total time: 0.00121501 s
File: /tmp/ipykernel_21621/1801258127.py
Function: pd2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def pd2(
     2                                               xyz: np.ndarray,
     3                                               model_params: dict[str, float | list[float]],
     4                                           ) -> dict[str, list[float]]:
     5                                               """
     6                                               if model params[key] is a single value return a linear dependent
     7                                               create a polynomial 0.0 * value * xyz[dependent_axis], that is
     8                                               parameter * axis position value for the  axis the parameter is dependent on
     9                                               if model params[key] is a list of values
    10

In [40]:
%timeit pd2(xyz_in, model)

251 μs ± 10.1 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [41]:
%prun pd2(xyz_in, model)

         659 function calls in 0.002 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       63    0.000    0.000    0.000    0.000 {built-in method numpy.array}
       21    0.000    0.000    0.001    0.000 polyutils.py:84(as_series)
        1    0.000    0.000    0.002    0.002 {built-in method builtins.exec}
       21    0.000    0.000    0.000    0.000 polynomial.py:664(polyval)
       21    0.000    0.000    0.000    0.000 type_check.py:682(common_type)
       21    0.000    0.000    0.001    0.000 _polybase.py:304(__init__)
       21    0.000    0.000    0.000    0.000 polyutils.py:260(mapparms)
       21    0.000    0.000    0.000    0.000 _polybase.py:510(__call__)
        1    0.000    0.000    0.001    0.001 1801258127.py:48(<dictcomp>)
        1    0.000    0.000    0.001    0.001 1801258127.py:1(pd2)
       42    0.000    0.000    0.000    0.000 polyutils.py:142(<genexpr>)
        1    0.000    0.000    0.000    

In [42]:
%timeit designl.linear_dependency(xyz_in, model)

6.09 μs ± 139 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [44]:
def pd3(
    xyz: np.ndarray,
    model_params: dict[str, float | list[float]],
) -> dict[str, list[float]]:
    """
    if model params[key] is a single value return a linear dependent
    create a polynomial 0.0 * value * xyz[dependent_axis], that is
    parameter * axis position value for the  axis the parameter is dependent on
    if model params[key] is a list of values
    calculate a polynomial form these coeficients and return
    the polnominal evaluated at the axis value for the dependent axis
    """
    x, y, z = xyz
    ld_dict = {
        "Txx": x,
        "Txy": x,
        "Txz": x,
        "Tyx": y,
        "Tyy": y,
        "Tyz": y,
        "Tzx": z,
        "Tzy": z,
        "Tzz": z,
        "Rxx": x,
        "Rxy": x,
        "Rxz": x,
        "Ryx": y,
        "Ryy": y,
        "Ryz": y,
        "Rzx": z,
        "Rzy": z,
        "Rzz": z,
        "Wxy": x,
        "Wxz": y,
        "Wyz": z,
    }
    pl = {}
    for key, value in model_params.items():
        if isinstance(value, list):
            poly = Polynomial(value)
        else:
            poly = Polynomial([0.0, value])
        pl[key] = poly(ld_dict[key])

    return pl

In [45]:
%timeit pd3(xyz_in, model)

297 μs ± 16.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [46]:
%lprun -f pd3 pd3(xyz_in, model)

Timer unit: 1e-09 s

Total time: 0.00062353 s
File: /tmp/ipykernel_21621/397047319.py
Function: pd3 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def pd3(
     2                                               xyz: np.ndarray,
     3                                               model_params: dict[str, float | list[float]],
     4                                           ) -> dict[str, list[float]]:
     5                                               """
     6                                               if model params[key] is a single value return a linear dependent
     7                                               create a polynomial 0.0 * value * xyz[dependent_axis], that is
     8                                               parameter * axis position value for the  axis the parameter is dependent on
     9                                               if model params[key] is a list of values
    10 

In [55]:
ballspacing = 133.0
x0, y0, z0 = 250.0, 50.0, 50.0
# XY plane
transform_mat_xy = dc.matrix_from_vectors([x0, y0, z0], [0.0, 0.0, 0.0])
xt, yt, zt = 0.0, 0.0, -243.4852
prb_xy = dc.Probe(title="P0", name="p0", length=np.array([xt, yt, zt]))
mmt_xy = dc.Measurement(
    title="Plate XY",
    name="mmt_00",
    artefact=cf.artefact_models["KOBA 0620"],
    transform_mat=transform_mat_xy,
    probe=prb_xy,
    cmm_nominal=None,
    cmm_dev=None,
    mmt_nominal=None,
    mmt_dev=None,
)
mmt_xy.recalculate(model, dc.pmm_866.cmm_model)

In [57]:
%timeit designl.linear_model_matrix(mmt_xy.cmm_nominal.T, mmt_xy.probe.length, model)

2.32 ms ± 91.5 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [58]:
%timeit designp.model_matrix(mmt_xy.cmm_nominal.T, mmt_xy.probe.length, model)

9.47 ms ± 446 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [59]:
%timeit designp.poly_dependency(xyz_in, model)

338 μs ± 38.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [61]:
%lprun -f  designp.model_point designp.model_matrix(mmt_xy.cmm_nominal.T, mmt_xy.probe.length, model)

Timer unit: 1e-09 s

Total time: 0.0433656 s
File: /home/elfnor/gits/ls_cmm_error_map/src/cmm_error_map/design_poly.py
Function: model_point at line 84

Line #      Hits         Time  Per Hit   % Time  Line Contents
    84                                           def model_point(
    85                                               xyz_in: np.ndarray,  # (3,)
    86                                               xyzt: np.ndarray,  # (3,)
    87                                               model_params: dict[str, list[float]],
    88                                               fixed_table=False,
    89                                               bridge_axis=1,
    90                                           ):
    91                                               """ """
    92        25      39982.0   1599.3      0.1      if not fixed_table:
    93                                                   # table movement is in opposite direction to probe position for a moving table
    9